In [1]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow.python.framework import ops
import math
from datetime import date
import requests
import json
import urllib
import sys
import Data_Collection_Processing_DNR_Prediction_Functions as script

C:\Users\gandh\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


X = Tensor("Placeholder:0", shape=(768, ?), dtype=float32)
Y = Tensor("Placeholder_1:0", shape=(3, ?), dtype=float32)
W1 = <tf.Variable 'W1:0' shape=(64, 768) dtype=float32_ref>
b1 = <tf.Variable 'b1:0' shape=(64, 1) dtype=float32_ref>
W2 = <tf.Variable 'W2:0' shape=(64, 64) dtype=float32_ref>
b2 = <tf.Variable 'b2:0' shape=(64, 1) dtype=float32_ref>
W3 = <tf.Variable 'W3:0' shape=(64, 64) dtype=float32_ref>
b3 = <tf.Variable 'b3:0' shape=(64, 1) dtype=float32_ref>
W4 = <tf.Variable 'W4:0' shape=(1, 64) dtype=float32_ref>
b4 = <tf.Variable 'b4:0' shape=(1, 1) dtype=float32_ref>


In [13]:
def Get_todays_Date():   
    today = str(date.today())
    today = today.split('-')
#     del(today[0])
    return str(today[0]) + str(today[1]) + str(today[2]) 

today = Get_todays_Date()
today

'20180831'

In [3]:
data = script.read_csv_file('Data/Neural Network/2015 to 2017 Solar with all Features.csv')
script.Check(data)

shape: (191800, 22)


,Latitude (°N),Longitude (°W),Relative Humidity at 2m (%),Surface Pressure (kPa),Temperature at 2m (℃),Downward Thermal Infrared Radiative Flux (kW-hr/m²/day),Clear Sky Insolation Clearness Index,Top of Atmosphere Insolation (kW-hr/m²/day),All Sky Insolation (kW-hr/m²/day),JAN,...,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,Direct Normal Radiation (kW-hr/m²/day)
0,32.25,124.25,72.23,102.08,15.17,72.23,0.40,5.16,2.04,1,...,0,0,0,0,0,0,0,0,0,3.413545
1,32.25,123.75,71.51,102.07,14.91,71.51,0.50,5.10,2.55,1,...,0,0,0,0,0,0,0,0,0,3.430104
2,32.25,123.25,70.56,102.05,14.72,70.56,0.50,5.10,2.55,1,...,0,0,0,0,0,0,0,0,0,3.420178
3,32.25,122.75,69.12,102.04,14.56,69.12,0.53,5.15,2.71,1,...,0,0,0,0,0,0,0,0,0,3.807574
4,32.25,122.25,67.54,102.03,14.49,67.54,0.53,5.15,2.71,1,...,0,0,0,0,0,0,0,0,0,3.836880


In [4]:
degree_celcius =  u"\u2103"
degree = u'\xb0'
square = '\u00b2'

In [5]:
# for i in range(5):
#     data['DNR T-' + str(i+1)] = data['Direct Normal Radiation (kW-hr/m' + square + '/day)'].shift(periods=i+1, freq=None, axis=0)
# data = data.dropna(axis=0, how='any').reset_index(drop=True)

# script.Check(data)

In [6]:
train_x, val_x, test_x, train_y, val_y, test_y = script.train_validate_test_split(data, 21, train_percent=.7, validate_percent=.2, seed=True)

names = ['train_x', 'val_x', 'test_x', 'train_y', 'val_y', 'test_y']
files = [train_x, val_x, test_x, train_y, val_y, test_y]

In [7]:
Normalized_files = []
Cols_max=[]
for file in files:
    Normalized_file = script.Normalize(file)[0]
    file_max = script.Normalize(file)[1]
    Normalized_files.append(Normalized_file)
    Cols_max.append(file_max)
    
for i in range(len(Normalized_files)):
    print(names[i]+ ': ', Normalized_files[i].shape)

train_x:  (134260, 21)
val_x:  (38360, 21)
test_x:  (19180, 21)
train_y:  (134260, 1)
val_y:  (38360, 1)
test_y:  (19180, 1)


In [8]:
# Saving Files as .txt

for i in range(6):
    print(names[i])
    script.Check(Normalized_files[i])
    np.savetxt('Data/Neural Network/Normalized Files/solar' + names[i] + '.txt', Normalized_files[i].values)
    np.savetxt('Data/Neural Network/Maximum Values/solar' + names[i] + '_max_values.txt', Cols_max[i].values)

train_x
shape: (134260, 21)
val_x
shape: (38360, 21)
test_x
shape: (19180, 21)
train_y
shape: (134260, 1)
val_y
shape: (38360, 1)
test_y
shape: (19180, 1)


In [9]:
solar_arr = []

for name in names:
    txt_file = np.loadtxt('Data/Neural Network/Normalized files/solar' + name + '.txt', dtype=float)
    txt_file = script.replace_zeros(txt_file)
    solar_arr.append(txt_file)

In [10]:
X_train = solar_arr[0]
X_val = solar_arr[1]
X_test = solar_arr[2]
y_train = solar_arr[3]
y_val = solar_arr[4]
y_test = solar_arr[5]

In [11]:
def create_placeholders(n_x, n_y):
    X = tf.placeholder(shape=[n_x, None], dtype=tf.float32)
    Y = tf.placeholder(shape=[n_y, None], dtype=tf.float32)
    return X, Y

X, Y = create_placeholders(20000, 6)
print ("X = " + str(X))
print ("Y = " + str(Y))

X = Tensor("Placeholder:0", shape=(20000, ?), dtype=float32)
Y = Tensor("Placeholder_1:0", shape=(6, ?), dtype=float32)


In [12]:
def initialize_parameters(n_x, n_y, layers=4, layers_dim=[500, 500, 500, 1]):
    
    tf.set_random_seed(1)
    parameters={}
    for l in range(layers):
        if l == 0:
            parameters["W"+str(l+1)] = tf.get_variable("W"+str(l+1), [layers_dim[l], n_x], initializer=tf.contrib.layers.xavier_initializer(seed = 1))
            parameters["b"+str(l+1)] = tf.get_variable("b"+str(l+1), [layers_dim[l], 1], initializer=tf.zeros_initializer())
        else:
            parameters["W"+str(l+1)] = tf.get_variable("W"+str(l+1), [layers_dim[l], layers_dim[l-1]], initializer=tf.contrib.layers.xavier_initializer(seed = 1))
            parameters["b"+str(l+1)] = tf.get_variable("b"+str(l+1), [layers_dim[l], 1], initializer=tf.zeros_initializer())
    
    return parameters

In [13]:
tf.reset_default_graph()
with tf.Session() as sess:
    parameters = initialize_parameters(20000, 6)
    print("W1 = " + str(parameters["W1"]))
    print("b1 = " + str(parameters["b1"]))
    print("W2 = " + str(parameters["W2"]))
    print("b2 = " + str(parameters["b2"]))
    print("W3 = " + str(parameters["W3"]))
    print("b3 = " + str(parameters["b3"]))
    print("W4 = " + str(parameters["W4"]))
    print("b4 = " + str(parameters["b4"]))

W1 = <tf.Variable 'W1:0' shape=(500, 20000) dtype=float32_ref>
b1 = <tf.Variable 'b1:0' shape=(500, 1) dtype=float32_ref>
W2 = <tf.Variable 'W2:0' shape=(500, 500) dtype=float32_ref>
b2 = <tf.Variable 'b2:0' shape=(500, 1) dtype=float32_ref>
W3 = <tf.Variable 'W3:0' shape=(500, 500) dtype=float32_ref>
b3 = <tf.Variable 'b3:0' shape=(500, 1) dtype=float32_ref>
W4 = <tf.Variable 'W4:0' shape=(1, 500) dtype=float32_ref>
b4 = <tf.Variable 'b4:0' shape=(1, 1) dtype=float32_ref>


In [14]:
def forward_propagation(X, parameters, activation_types=["relu","relu","relu","relu"]):
    mul = {}
    L = int(len(parameters)/2)
    for l in range(L):
        if l == 0:
            mul["Z"+str(l+1)] = parameters["W"+str(l+1)] @ X + parameters["b"+str(l+1)]
        else:
            mul["Z"+str(l+1)] = parameters["W"+str(l+1)] @ mul["A"+str(l)] + parameters["b"+str(l+1)]
        if activation_types[l] == "relu":
            mul["A"+str(l+1)] = tf.nn.relu(mul["Z"+str(l+1)])
        elif activation_types[l] == "sigmoid":
            mul["A"+str(l+1)] = tf.nn.sigmoid(mul["Z"+str(l+1)])
        else:
            mul["A"+str(l+1)] = mul["Z"+str(l+1)]
        
    return mul["A"+str(int(len(parameters)/2))]
    

In [15]:
def compute_cost(AL, Y, cost_type="mse"):
#     m = Y.shape[0]
    cost = tf.reduce_mean((AL-Y)**2)
    return cost

In [16]:
def random_minibatches(X, Y, mini_batch_size=8, seed=1):
    np.random.seed(seed)            # To make your "random" minibatches the same as ours
    m = X.shape[1]                  # number of training examples
    mini_batches = []
        
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1,m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X = shuffled_X[:, k*mini_batch_size: (k+1)*mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k*mini_batch_size: (k+1)*mini_batch_size]
        ### END CODE HERE ###
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches:]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [27]:
def model(X_train, Y_train, X_test, Y_test, learning_rate=0.01, num_epochs=100, print_cost=True, mini_batch_size=8):
    ops.reset_default_graph()
#     tf.set_random_seed(1)
    (n_x, m) = X_train.shape
    n_y = Y_train.shape[0]
    costs=[]
    
    X, Y = script.create_placeholders(n_x, n_y)
    
    parameters = initialize_parameters(n_x, n_y)
    
    AL = forward_propagation(X, parameters)
    
    cost = compute_cost(AL, Y)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            epoch_cost = 0.
            num_minibatches = m/mini_batch_size
            minibatches = random_minibatches(X_train, Y_train)
            for minibatch in minibatches:
                minibatch_X, minibatch_Y = minibatch
                _, minibatch_cost = sess.run((optimizer, cost), feed_dict={X:minibatch_X, Y:minibatch_Y})
                epoch_cost += minibatch_cost/num_minibatches
                
            if print_cost == True:
                if epoch%50==0:
                    costs.append(epoch_cost)
                    if epoch%100==0:
                        print("Cost after epoch %i: %f" % (epoch, epoch_cost))
                        
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        parameters = sess.run(parameters)
        accuracy = tf.reduce_mean(1-AL/Y)

        print("Train accuracy: ", accuracy.eval({X: X_train, Y: Y_train}))
        print("Test accuracy: ", accuracy.eval({X: X_test, Y: Y_test}))
    
    return parameters

parameters = model(X_train, y_train, X_val, y_val)

IndexError: too many indices for array

In [26]:
x = [1,0,2,90,0.00001, 0, -87, -90, 674]
np.maximum(x,0)

array([1.00e+00, 0.00e+00, 2.00e+00, 9.00e+01, 1.00e-05, 0.00e+00,
       0.00e+00, 0.00e+00, 6.74e+02])

In [45]:
def Get_Yesterday():
    yes = str(date.today() + timedelta(days=-1)).split('-')
    return str(yes[0] + yes[1] + yes[2])

In [59]:
California_bbox = [32.5, -124.5, 42, -114.1]
California_bbox_list = script.split_bbox(California_bbox)
solar_features = script.create_features(['PRECTOT', 'RH2M', 'PS', 'T2M', 'ALLSKY_SFC_LW_DWN', 'KT_CLEAR', 'CLRSKY_SFC_SW_DWN', 'ALLSKY_TOA_SW_DWN', 'ALLSKY_SFC_SW_DWN'])
dates = script.create_Dates(Get_Yesterday(), Get_Yesterday())

In [60]:
def Download_Data(bbox_list, features, dates, user_community, temp_Average, Folder_name):
    for j in range(len(bbox_list)):
        url = 'https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?request=execute&identifier=Regional&parameters='\
        + features + dates + \
        '&userCommunity=' + user_community + '&tempAverage=' + temp_Average + '&outputList=CSV&bbox=' +\
        str(California_bbox_list[j][0]) + ',' + str(bbox_list[j][1]) + ',' + str(bbox_list[j][2]) + ',' + \
        str(bbox_list[j][3]) + '&user=anonymous'
        response = requests.get(url)
        csv_url = response.json()['outputs']['csv']
        urllib.request.urlretrieve(csv_url, 'Data/' + Folder_name + '/File_' + str(j+1) + '.csv')
        print('set ' + str(j+1) + ' is complete' )

In [61]:
Download_Data(California_bbox_list, solar_features, dates, 'SB', 'DAILY', 'Today')

set 1 is complete
set 2 is complete
set 3 is complete


In [33]:
script.Get_csv(California_bbox_list, 'Today', 21)

TypeError: Get_csv() missing 1 required positional argument: 'header'